In [10]:
import pandas as pd
import numpy as np
import pymongo
import requests
import json
import pickle
from collections import defaultdict
from collections import Counter

In [5]:
client = pymongo.MongoClient("mongodb://localhost:37017" )
db=client["lol"]
coll=db["match_data"]

In [73]:
def pull_champions_from_game_data(mongo_object):
    """Takes a dictionary stored in my mongodb.
    Returns ordered lists of allied champions,
    enemy champions, and the winning team id as an int.
    """
    champs=[]
    winning_team=0
    for player in mongo_object["data"]["participants"]:
        champs.append(player["championId"])
    allied_champions=champs[:5]
    enemy_champions=champs[5:]
    if mongo_object["data"]["teams"][0]["win"]=="Fail":
        winning_team=100
    else:
        winning_team=200
    return allied_champions,enemy_champions,winning_team
        

In [12]:
champion_vs=Counter()

In [84]:
def count_matchups(allied_champions,enemy_champions,winning_team):
    """
    Returns counter dict of how many times [key] matchup has occured
    (4,7):3 == champion 4 has *matched against* champion 7 three times
    and
    Returns counter dict of how many times [key] matchup has resulted in a win
    (4,7):3 == champion 4 has *won against* champion 7 three times
    """
    matchups=Counter()
    matchups_wins=Counter()
    for champ in allied_champions:
        for enemychamp in enemy_champions:
            matchups[(champ,enemychamp)]+=1
            if winning_team==100:
                matchups_wins[(champ,enemychamp)]+=1
    for champ in enemy_champions:
        for enemychamp in allied_champions:
            matchups[(champ,enemychamp)]+=1
            if winning_team==200:
                matchups_wins[(champ,enemychamp)]+=1
    return matchups, matchups_wins

In [85]:
# def count_matchups_wins(allied_champions,enemy_champions,winning_team):
#     """Returns counter dict of how many times [key] matchup has resulted in a win
#     (4,7):3 == champion 4 has *won against* champion 7 three times
#     """
#     matchups=Counter()
#     for champ in allied_champions:
#         for enemychamp in enemy_champions:
#             if winning_team==100:
#                 matchups[(champ,enemychamp)]+=1
#     for champ in enemy_champions:
#         for enemychamp in allied_champions:
#             if winning_team==200:
#                 matchups[(champ,enemychamp)]+=1
#     return matchups
    

In [86]:
def total_and_wins_for_matchup(iterable_games_data):
    """
    Calls and aggregates results of count_matchups() over 
    iterable collection of games data.
    Returns two counter dicts
    """
    total_vs=Counter()
    wins_vs=Counter()
    for game in iterable_games_data:
        matchups, matchups_wins=count_matchups(*pull_champions_from_game_data(game))
        total_vs+=matchups
        wins_vs+=matchups_wins
    return total_vs,wins_vs

In [87]:
total, wins = total_and_wins_for_matchup(coll.find({"done":"true"}).limit(20))

In [88]:
sum(total.values()),sum(wins.values())

(1000, 500)

In [89]:
def count_synergies(allied_champions,enemy_champions,winning_team):
    """
    Returns counter dict of how many times [key] synergy has occured
    (4,7):3 == champion 4 has *matched with* champion 7 three times
    and
    Returns counter dict of how many times [key] synergy has resulted in a win
    (4,7):3 == champion 4 has *won with* champion 7 three times
    """
    synergy=Counter()
    synergy_wins=Counter()
    for i,champ in enumerate(allied_champions):
        for i2,champ2 in enumerate(allied_champions):
            if i==i2:
                continue
            synergy[(champ,champ2)]+=1
            if winning_team==100:
                synergy_wins[(champ,champ2)]+=1
    for i,champ in enumerate(enemy_champions):
        for i2,champ2 in enumerate(enemy_champions):
            if i==i2:
                continue
            synergy[(champ,champ2)]+=1
            if winning_team==200:
                synergy_wins[(champ,champ2)]+=1
    return synergy,synergy_wins

In [ ]:
# def count_synergies_wins(allied_champions,enemy_champions,winning_team):
#     """Returns counter dict of how many times [key] synergy has resulted in a win
#     (4,7):3 == champion 4 has *won with* champion 7 three times
#     """
#     synergy=Counter()
#     for i,champ in enumerate(allied_champions):
#         for i2,champ2 in enumerate(allied_champions):
#             if i==i2:
#                 continue
#             if winning_team==100:
#                 synergy[(champ,champ2)]+=1
#     for i,champ in enumerate(enemy_champions):
#         for i2,champ2 in enumerate(enemy_champions):
#             if i==i2:
#                 continue
#             if winning_team==200:
#                 synergy[(champ,champ2)]+=1
#     return synergy
    

In [81]:
def total_and_wins_for_synergy(iterable_games_data):
    """
    Calls and aggregates results of count_synergies() over 
    iterable collection of games data.
    Returns two counter dicts
    """
    total_synergy=Counter()
    wins_synergy=Counter()
    for game in iterable_games_data:
        synergy,synergy_wins=count_synergies(*pull_champions_from_game_data(game))
        total_synergy+=synergy
        wins_synergy+=synergy_wins
    return total_synergy,wins_synergy

In [82]:
total_s,wins_s=total_and_wins_for_synergy(coll.find({"done":"true"}).limit(20))

In [83]:
sum(total_s.values())

800